In [1]:
import pandas as pd
import re
import requests
import json
import functools
from pprint import pprint
from docx import Document
from os import listdir, environ
from os.path import isfile, join, abspath
from collections import defaultdict

In [2]:

datafiles = [f for f in listdir('./data/') if isfile(join('./data/', f))]
columns = ['Дата', 'Адрес', 'Оксид углерода', 'Оксид азота', 'Диоксид азота', 'Диоксид серы', 'Озон', 'Взвешенные частицы PM10', 'Взвешенные частицы PM2.5']

In [3]:
def addDate(fn):
    df = pd.DataFrame(columns=columns)
    with open(abspath('data/'+fn), 'rb') as f:
        pollution_report = Document(f)

        adr_cnt = 0
        for i, paragraph in enumerate(pollution_report.paragraphs):
            row_df = dict.fromkeys(columns)
            adress = re.findall("адресу: (.*?)\\(станции", paragraph.text)
            if adress:
                row_df['Адрес'] = adress[0]
                row_df['Дата']  = fn.split('.')[0]
                for j in range(1, len(pollution_report.tables[adr_cnt].rows)):
                    row = pollution_report.tables[adr_cnt].rows[j]
                    row_text = []
                    for cell in row.cells:
                        row_text.append(cell.text)
                    row_df[row_text[0]] = row_text[1]
                df = df.append(row_df, ignore_index=True)
                adr_cnt += 1
    return df

In [4]:
# Construct dataframe from all of the documents
df = pd.DataFrame(columns=columns)

for file in datafiles:
    df = df.append(addDate(file), ignore_index=True)

In [5]:
# Filter out adresses for better mapbox search
def filter_adress(adr):
    return functools.reduce(lambda x, y: x.replace(y, abbr[y]), abbr, adr).rstrip()

abbr = {'пр.':'проспект', 'ул.': 'улица', 'В.О.': '', 'пер.': 'переулок', 'г.': 'город', 'пос.': 'поселок'}
df['Адрес'] = df['Адрес'].map(filter_adress)
adresses = df['Адрес'].unique()
adresses

array(['Московский проспект, дом 19', 'Средний проспект, дом 74',
       'проспект КИМа, дом 26 лит. А', 'улица Карбышева, дом 7',
       'улица Ольги Форш, дом 6', 'город Колпино, Красная улица, дом 1а',
       'поселок Металлострой, Железнодорожная улица, дом 13',
       'Малоохтинский проспект, дом 98', 'Индустриальный проспект, д.64',
       'Уткин проспект, д.16', 'Волхонское шоссе, д.116, корп. 3',
       'город Кронштадт, улица Ильмянинова, дом 4',
       'город Сестрорецк, улица М. Горького, дом 2',
       'улица Севастьянова, дом 11', 'улица Тельмана, дом 24',
       'улица Профессора Попова, дом 48',
       'город Ломоносов, улица Федюнинского, дом 3',
       'улица Новосельковская, дом 23',
       'город Пушкин, Тиньков переулок, дом 4',
       'Малая Балканская улица, дом 54', 'Шпалерная улица, дом 56',
       'улица Пестеля, дом 1', 'проспект Маршала Жукова, дом 30, корп. 3'],
      dtype=object)

In [6]:
# Get geo-coordinates for each unique adress
bbox = '30.0874,59.8003,30.556,60.07'
access_token = 'pk.eyJ1IjoiYWxleDQ5MiIsImEiOiJjank1b3BleXcwOHM4M29tbHY1bWVhaXA1In0.HY6gliRl51T_WMSxdOGbJw'
base_url = "https://api.mapbox.com/geocoding/v5/mapbox.places/{}.json?bbox={}&access_token={}"

georef = dict.fromkeys(adresses)

for adress in adresses:
    search_result = requests.get(base_url.format(adress, bbox, access_token)).json()
    georef.update({adress: search_result['features'][0]['center']})

In [28]:
df['lon'] = df['Адрес'].apply(lambda row: georef[row][0])
df['lat'] = df['Адрес'].apply(lambda row: georef[row][1])

In [29]:
df

,Дата,Адрес,Оксид углерода,Оксид азота,Диоксид азота,Диоксид серы,Озон,Взвешенные частицы PM10,Взвешенные частицы PM2.5,lat_lon,lon,lat
0,18_07_2019,"Московский проспект, дом 19",0.1,менее 0.1,0.6,менее 0.1,None,None,None,"[30.319603, 59.872354]",30.319603,59.872354
1,18_07_2019,"Средний проспект, дом 74",0.1,менее 0.1,0.4,None,1.1,None,0.3,"[30.2745135, 59.9424645]",30.274514,59.942464
2,18_07_2019,"проспект КИМа, дом 26 лит. А",0.1,0.1,0.6,0.1,None,None,None,"[30.2443917, 59.9522399]",30.244392,59.952240
3,18_07_2019,"улица Карбышева, дом 7",0.1,0.1,0.5,None,None,0.2,None,"[30.329922, 59.931213]",30.329922,59.931213
4,18_07_2019,"улица Ольги Форш, дом 6",0.1,0.2,0.5,None,1.0,0.1,None,"[30.220726, 59.995415]",30.220726,59.995415
5,18_07_2019,"город Колпино, Красная улица, дом 1а",0.1,0.1,0.3,0.1,None,0.6,None,"[30.3207273, 59.9140252]",30.320727,59.914025
6,18_07_2019,"поселок Металлострой, Железнодорожная улица, д...",0.1,0.1,0.5,менее 0.1,0.7,0.2,None,"[30.5516319, 59.805144]",30.551632,59.805144
7,18_07_2019,"Малоохтинский проспект, дом 98",менее 0.1,менее 0.1,0.2,менее 0.1,1.5,0.1,None,"[30.3996642, 59.9297239]",30.399664,59.929724
8,18_07_2019,"Индустриальный проспект, д.64",0.1,0.1,0.4,менее 0.1,None,0.1,None,"[30.4660558, 59.9522512]",30.466056,59.952251
9,18_07_2019,"Уткин проспект, д.16",0.1,менее 0.1,0.4,менее 0.1,None,0.2,None,"[30.4374322, 59.9319429]",30.437432,59.931943


In [9]:
import plotly
import plotly.offline as offline
import plotly.graph_objs as go

In [32]:
#10.31 59.939,30.344
custom_style = "mapbox://styles/alex492/cjyko598j0o2u1cknj8yazniz"
custom_style = "mapbox://styles/alex492/cjyil3dry03rl1cqr4mlut05k"

data = [go.Scattermapbox(
            lat= df['lat'],
            lon= df['lon'],
            #customdata = fix_stat_dfs[0][1],
            mode='markers',
            marker=dict(
                size= 15,
                color = 'gold',
                opacity = .8,
            ),
            name='novatel'
          )]
layout = go.Layout(autosize=False,
                   mapbox= dict(accesstoken=access_token,
                                bearing=10,
                                pitch=60,
                                zoom=11,
                                center= dict(lat=59.939,
                                             lon=30.344),
                                style=custom_style),
                    width=900,
                    height=600, 
                    title = "gps_raw_record")

In [33]:
fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig)

pk.eyJ1IjoiYWxleDQ5MiIsImEiOiJjank1b3BleXcwOHM4M29tbHY1bWVhaXA1In0.HY6gliRl51T_WMSxdOGbJw
